In [ ]:
# !pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Assignment2 Spark loops").getOrCreate()
sc = spark.sparkContext


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# rdd = sc.textFile("/content/drive/MyDrive/pagecounts-20160101-000000_parsed.out")
# print(rdd.first())  # just to make sure the file is read


rdd = sc.textFile("work/FCAI/BigData/spark-ass2/pagecounts-20160101-000000_parsed.out")
print(rdd.first())  # just to make sure the file is read

aa 271_a.C 1 4675


In [3]:
import time
import re

def parse_line(line):
    try:
        parts = line.strip().split(' ')
        return (parts[0], parts[1], int(parts[2]), int(parts[3]))
    except:
        return None

parsed_rdd = rdd.map(parse_line).filter(lambda x: x is not None)
parsed_rdd.persist()


# Q1: page size stats
start_time = time.time()

sizes = parsed_rdd.map(lambda line: line[3]).filter(lambda x: x >= 0).collect()

min_size = min(sizes)
max_size = max(sizes)
avg_size = sum(sizes) / len(sizes) if sizes else 0

print("Q1 (Loops) \t min size:", min_size, "\t Max size:", max_size, "\t Avg size:", avg_size)
print("Q1 (Loops) \t total time in seconds:", time.time() - start_time, "s")


# Q2: titles starting with 'The' & not 'en'
start_time = time.time()

lines = parsed_rdd.collect()

the_titles = [line for line in lines if line[1].startswith("The")]
non_en_the = [line for line in the_titles if line[0] != "en"]

print("\n\n")
print("Q2 (Loops) \t total titles starting with 'The':", len(the_titles))
print("Q2 (Loops) \t total non-English titles starting with 'The':", len(non_en_the))
print("Q2 (Loops) \t total time in seconds:", time.time() - start_time, "s")


# Q3: Unique terms in page_title
start_time = time.time()

def normalize_term(term):
    return re.sub(r'\W+', '', term.lower())

titles = parsed_rdd.map(lambda x: x[1]).collect()

terms_set = set()
for title in titles:
    terms = title.split('_')
    for term in terms:
        normalized = normalize_term(term)
        if normalized:
            terms_set.add(normalized)

print("\n\n")
print("Q3 (Loops) \t total unique normalized terms:", len(terms_set))
print("Q3 (Loops) \t sample terms:", list(terms_set)[:5])
print("Q3 (Loops) \t total time in seconds:", time.time() - start_time, "s")


# Q4: Count titles
start_time = time.time()

titles = parsed_rdd.map(lambda x: x[1]).collect()

title_count_dict = {}
for title in titles:
    title_count_dict[title] = title_count_dict.get(title, 0) + 1

print("\n\n")
print("Q4 (Loops) \t title count sample:", list(title_count_dict.items())[:5])
print("Q4 (Loops) \t total time in seconds:", time.time() - start_time, "s")



# Q5: Group lines by title
start_time = time.time()

lines = parsed_rdd.collect()

combined_dict = {}
for line in lines:
    project, title, hits, size = line
    if title not in combined_dict:
        combined_dict[title] = []
    combined_dict[title].append((project, hits, size))

print("\n\n")
print("Q5 (Loops) \t sample combined titles:")
for k, v in list(combined_dict.items())[:5]:
    print(f"{k} -> {v}")

print("Q5 (Loops) \t total time in seconds:", time.time() - start_time, "s")



Q1 (Loops) 	 min size: 0 	 Max size: 141180155987 	 Avg size: 132239.56957446598
Q1 (Loops) 	 total time in seconds: 5.116133689880371 s



Q2 (Loops) 	 total titles starting with 'The': 45020
Q2 (Loops) 	 total non-English titles starting with 'The': 10292
Q2 (Loops) 	 total time in seconds: 2.8902223110198975 s



Q3 (Loops) 	 total unique normalized terms: 1689811
Q3 (Loops) 	 sample terms: ['ekel', 'specialexportabbottabad', 'specialentitydataq5934804json', 'talkmaximumgrosstakeoffweight', 'fileenusspoilsportogg']
Q3 (Loops) 	 total time in seconds: 10.373650074005127 s



Q4 (Loops) 	 title count sample: [('271_a.C', 4), ('Category:User_th', 2), ('Chiron_Elias_Krase', 6), ('Dassault_rafaele', 3), ('E.Desv', 6)]
Q4 (Loops) 	 total time in seconds: 3.753314256668091 s



Q5 (Loops) 	 sample combined titles:
271_a.C -> [('aa', 1, 4675), ('az', 1, 6356), ('bcl', 1, 5068), ('be', 1, 6287)]
Category:User_th -> [('aa', 1, 4770), ('commons.m', 1, 0)]
Chiron_Elias_Krase -> [('aa', 1, 4694)